# Tutorial para acceso a datos del Banco Nacional de Información Hidrometeorológico BANADIH. V.1.0.0 

| Fecha de Actualización | Descripción de la Actualización                                | Responsable          |
|------------------------|---------------------------------------------------------------|----------------------|
| 2025-04-26             | Se añade la libreria de conexion al Banco Nacional de Información Hidroemteorológica __BANADIH__ | Darwin Rosero           |
| 2025-04-29             | Ejemplos de consultas básicas | Darwin Rosero           |
| ******            | ****** | ******           |

El siguiente libro sirve como ejemplo para consultar datos hidrometeorológicos en la infraestructura del INAMHI.



# Instalación de LaTeX en el entorno del usuario  

Si se queire activar el compilador de LaTeX, en el usuario de debera ejecutar la sigueinte linea de código en el usuario 
```python 
!pip install jupyterlab-latex
```



# 1. Importante! Sección de librerías.
Esta sección es para importar todas las librerías necesarías, aqui se deben realizar todos las importaciones, la librería que es obligatoría si se quiere consultar datos desde el __BAnco NAcional De Información Hidrometeorológica__. es la siguiente:
```python
from banadih import con_db
```
Tambien se deben importar aquí todas las librerías que se requieran para ejecutar el script.

In [1]:
# Importamos la librería que permite conectarse al Baknco Nacional de Información Hidrometeorlógica
from banadih import con_db 
# a continuación importar todas las demas librerías.
import pandas as pd
pd.set_option('display.max_rows', 1000)  # Muestra hasta 1000 filas
pd.set_option('display.max_columns', 1000)  # Muestra hasta 1000 columnas
# libretia para manejar fechas 
from datetime import datetime, timedelta

## 2 Modulos del __BAnco NAcional De Información Hidrometeorológica__
El BANADIH esta dividido en tres módulos, cada uno cumple una función especifica y tinene un enfoque especifico de uso, a continuacion se hace una descripción de cada módulo 

**Módulo de tiempo real**. __paramsRT__ Contiene acceso a los datos de las observaciones en tiempo real, con la menor temporalidad posible para cada estación (5 minutos), se puede consultar continuamente, los datos meteorológicos e hidrológicos incluyen mediciones estadísticas máximas, mínimas, instantáneos, sumas y promedio de las estaciones, tanto propietarias como con estaciones externas que envían datos a nuestros servidores.

**Módulo de datos históricos.** __paramsHIST__ Contiene los datos históricos de las estaciones meteorológicas e hidrológicos existentes a nivel horario, adicional esta contiene toda la Metadata necesaria para gestionar la información a nivel nacional, se puede consultar información desde el inicio de la operación de la estación registrado en la BANADIH.

**Módulo de datos agregados o públicos.** __paramsPublic__. Contiene las agregaciones de las observaciones; datos diarios, decadales (cada 10 días), mensuales, anuales e índices climáticos, estos datos se entregan al público y son visualizados a través de los visores institucionales.

**Módulo de datos Pronósticos.** __paramsPronos__. Contiene los valores pronosticados de la modelación númerica generada con el modelo wrf, extraido desde lso archivos tif generados diariamente.##

Para realizar la consultas a los modulo del BANADIH es necesario especificar el módulo de consulta que se pretende utilizar por ejemplo si se queire consultar datos en **tiempo real** la función quedaria como lo siguiente:

``` python

con_db.query_to_dataframe(query, params=con_db.paramsPublic)
```

In [16]:
## 1. Escribimos lso códigos de las estaciones que deseamos consultar.
codigos = ('M0001', 'M0002', 'M0002', 'M0004', 'M0258', 'M1036', 'M0008', 'M0031')

## 2. Creamos la cunsulta de base de datos en lenguaje SQL. (Solicitar soporte a la DIH)
query = f"""select id_estacion, codigo, punto_obs,latitud, longitud, altitud ,provincia,
canton, captor
from administrativo.vta_estaciones where codigo in {codigos}"""

## 3. Llamamos a la funcion que nos permitira retornar los datos en un dataframe de pandas (objeto python)
# Ejecutar la consulta y almacenar los resultados en un DataFrame

estaciones = con_db.query_to_dataframe(query, params=con_db.paramsHIST)

display(estaciones)

,id_estacion,codigo,punto_obs,latitud,longitud,altitud,provincia,canton,captor
0,26,M0004,RUMIPAMBA SALCEDO,-1.020000,-78.594600,2685.00,COTOPAXI,SALCEDO,ELECTROMECANICA
1,63758,M0004,RUMIPAMBA SALCEDO,-1.020000,-78.594600,2685.00,COTOPAXI,SALCEDO,MANUAL
2,23,M0031,CAÑAR,-2.552200,-78.945200,3083.00,CAÑAR,CAÑAR,ELECTROMECANICA
3,63760,M0031,CAÑAR,-2.552200,-78.945200,3083.00,CAÑAR,CAÑAR,MANUAL
4,12,M0002,LA TOLA,-0.231789,-78.370433,2493.00,PICHINCHA,QUITO,ELECTROMECANICA
5,63757,M0002,LA TOLA,-0.231789,-78.370433,2493.00,PICHINCHA,QUITO,MANUAL
6,13,M0258,QUEROCHACA,-1.367100,-78.605539,2865.00,TUNGURAHUA,CEVALLOS,ELECTROMECANICA
7,63759,M0258,QUEROCHACA,-1.367100,-78.605539,2865.00,TUNGURAHUA,CEVALLOS,MANUAL
8,63767,M0008,PUYO,-1.505700,-77.956000,956.00,PASTAZA,PASTAZA,MANUAL
9,2,M0008,PUYO,-1.505700,-77.956000,956.00,PASTAZA,PASTAZA,ELECTROMECANICA


## 3.2 Consulta de parámetros existentes en base de datos con sus ***NEMONICOS***
En este ejemplo consultaremos la parámetros disponibles en base de datos con sus NEMONICOS, que son los nombres estandarizados que damos a las variables observadas, para el caso de observaciones todos los nemonicos se definiran a escala de 1 hora, es decir, **por ejemplo las observaciones de las estaciones convencionales de las horas principales a las 07, 10, 13, 16, 17 horas respectivamente seran consideradas observaciones horarias**,
 
Asi como las observaciones que se realicen las 24 horas del día tambien se consideran como observaciones de una hora por tanto lso nemónicos terminaran en **1h**

en cuanto a las observaciones de las estaciones automáticas, por facilidad de nomenclatura, tambien se consideran de una hora, por lo que sus nemonicos terminan en **1h**


In [7]:
# consulta de parametrosabs
query = """select nemonico_2, parametro, unidad_medida, estadistico,unidad_medida_tiempo,
intervalo_tiempo, nemonico from administrativo.vta_parametros 
where unidad_medida_tiempo = 'HORAS' """
## 3. Llamamos a la funcion que nos permitira retornar los datos en un dataframe de pandas (objeto python)
# Ejecutar la consulta y almacenar los resultados en un DataFrame

parametros = con_db.query_to_dataframe(query, params=con_db.paramsHIST)

parametros

,nemonico_2,parametro,unidad_medida,estadistico,unidad_medida_tiempo,intervalo_tiempo,nemonico
0,146060101h,ALUMINIO,h,MAX,HORAS,1,146060101h
1,146121601h,ALUMINIO,mg/L,INST,HORAS,1,146121601h
2,132041601h,CALIDAD DEL DATO,Adimensional,INST,HORAS,1,132041601h
3,071271601h,CANTIDAD TOTAL NUBES,OCT,INST,HORAS,1,071271601h
4,126300101h,CAUDAL,m3/s,MAX,HORAS,1,1263011h
5,126301601h,CAUDAL,m3/s,INST,HORAS,1,12630161h
6,126300401h,CAUDAL,m3/s,PROM,HORAS,1,1263041h
7,126300201h,CAUDAL,m3/s,MIN,HORAS,1,1263021h
8,133301601h,CAUDAL DE AGUA OBSERVADO,m3/s,INST,HORAS,1,13330161h
9,138351601h,CIANOBACTERIAS,m cell/ml,INST,HORAS,1,138351601h


## 3.3 Filtros de parámetros.
En este ejemplo se realizara filtros para seleccionar aquellos parámetros que concuerden con una descripción.


In [8]:
# filtro al objeto parametros

variable = "temperatura aire"

mascara = parametros['parametro'].str.contains(variable,  case=False, na=False)
#    case=False: Hace la búsqueda insensible a mayúsculas/minúsculas (opcional)
#    na=False:   Trata los valores NaN/NA como si no contuvieran la cadena (devuelve False para ellos)
# Aplica la máscara para filtrar el DataFrame
parametros_filtrado = parametros[mascara]
display(parametros_filtrado)

,nemonico_2,parametro,unidad_medida,estadistico,unidad_medida_tiempo,intervalo_tiempo,nemonico


## 3.4 Consusltas de variables por estación al BANADIH.
El siguiente ejempo consulta los parametros o variables disponibles por estacion, agergando la fecha del primer dato disponible y la fecha del último dato, asi como si esta operativa o no la estación.


In [9]:
query_info_stations = f"""
        select
        	ve.codigo ,ve.id_estacion , ve.estado_estacion ,
        	ve.categoria, ep.captor, ep.nemonico ,
        	vp.parametro, vp.unidad_medida, vp.estadistico,
        	vp.unidad_medida_tiempo, ep.fecha_primer_dato , ep.fecha_ultimo_dato ,
        	ep.numero_datos 
        from
        	administrativo.vta_estaciones ve,
        	administrativo.estaciones_parametros ep,
        	administrativo.vta_parametros vp
        where
        	ve.id_estacion = ep.id_estacion
        	and vp.nemonico_2 = ep.nemonico
        order by ve.codigo, ep.nemonico
    """
df = con_db.query_to_dataframe(query_info_stations)
display(df)

,codigo,id_estacion,estado_estacion,categoria,captor,nemonico,parametro,unidad_medida,estadistico,unidad_medida_tiempo,fecha_primer_dato,fecha_ultimo_dato,numero_datos
0,0051020010,66305,OPERATIVA,METEOROLOGICA,ELECTROMECANICA,014101601h,NIVEL DEL AGUA,m,INST,HORAS,NaT,NaT,0.0
1,0051020010,66305,OPERATIVA,METEOROLOGICA,MANUAL,017140801h,PRECIPITACION,mm,SUM,HORAS,NaT,NaT,0.0
2,0051020010,66305,OPERATIVA,METEOROLOGICA,ELECTROMECANICA,029031601h,TEMPERATURA DEL AIRE,°C,INST,HORAS,NaT,NaT,0.0
3,0051020040,66306,OPERATIVA,METEOROLOGICA,ELECTROMECANICA,014101601h,NIVEL DEL AGUA,m,INST,HORAS,NaT,NaT,0.0
4,0051020040,66306,OPERATIVA,METEOROLOGICA,ELECTROMECANICA,017140801h,PRECIPITACION,mm,SUM,HORAS,NaT,NaT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12358,VM01-ARG,64391,EN MANTENIMIENTO,METEOROLOGICA,MANUAL,059071601h,TENSION DE VAPOR DEL AGUA,hPa,INST,HORAS,2024-07-25 03:20:31.476679,2024-07-25 03:20:31.476679,1.0
12359,VM01-ARG,64391,EN MANTENIMIENTO,METEOROLOGICA,MANUAL,060031601h,PUNTO DE ROCIO,°C,INST,HORAS,2024-07-25 03:20:31.476679,2024-07-25 03:20:31.476679,1.0
12360,VM01-ARG,64391,EN MANTENIMIENTO,METEOROLOGICA,MANUAL,072291601h,VISIBILIDAD HORIZONTAL PREVALECIENTE,km,INST,HORAS,2024-07-25 03:20:31.476679,2025-04-30 16:05:53.495120,6.0
12361,VM01-ARG,64391,EN MANTENIMIENTO,METEOROLOGICA,MANUAL,073041601h,FENOMENOS METEOROLOGICOS,Adimensional,INST,HORAS,2024-03-25 15:56:51.663826,2025-04-30 16:05:53.495120,307.0


In [10]:
# Definir las palabras clave a buscar en la columna 'parametro'
keywords = ["temperatura"]

# Filtrar el DataFrame por las palabras clave y donde 'numero_datos' no sea nulo
filtered_df = df[df['parametro'].str.contains('|'.join(keywords), case=False, na=False) & df['numero_datos'].notna()]

# Mostrar los resultados filtrados
filtered_df


,codigo,id_estacion,estado_estacion,categoria,captor,nemonico,parametro,unidad_medida,estadistico,unidad_medida_tiempo,fecha_primer_dato,fecha_ultimo_dato,numero_datos
2,0051020010,66305,OPERATIVA,METEOROLOGICA,ELECTROMECANICA,029031601h,TEMPERATURA DEL AIRE,°C,INST,HORAS,NaT,NaT,0.0
5,0051020040,66306,OPERATIVA,METEOROLOGICA,ELECTROMECANICA,029031601h,TEMPERATURA DEL AIRE,°C,INST,HORAS,NaT,NaT,0.0
8,0051025090,66300,OPERATIVA,METEOROLOGICA,ELECTROMECANICA,029031601h,TEMPERATURA DEL AIRE,°C,INST,HORAS,NaT,NaT,0.0
13,0051027050,66283,OPERATIVA,HIDROLOGICA,ELECTROMECANICA,029031601h,TEMPERATURA DEL AIRE,°C,INST,HORAS,NaT,NaT,0.0
16,0051027060,66288,OPERATIVA,HIDROLOGICA,ELECTROMECANICA,029031601h,TEMPERATURA DEL AIRE,°C,INST,HORAS,NaT,NaT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12341,V_M0032,65003,EN MANTENIMIENTO,METEOROLOGICA,MANUAL,029031601h,TEMPERATURA DEL AIRE,°C,INST,HORAS,2024-09-22 11:32:36.405655,2025-04-09 08:56:06.992427,105.0
12353,VM01-ARG,64391,EN MANTENIMIENTO,METEOROLOGICA,MANUAL,029030101d,TEMPERATURA DEL AIRE,°C,MAX,DIAS,2024-09-29 23:28:41.240338,2025-04-30 16:05:53.495120,6.0
12354,VM01-ARG,64391,EN MANTENIMIENTO,METEOROLOGICA,MANUAL,029030201d,TEMPERATURA DEL AIRE,°C,MIN,DIAS,2024-09-29 23:28:41.240338,2025-04-30 16:05:53.495120,8.0
12355,VM01-ARG,64391,EN MANTENIMIENTO,METEOROLOGICA,MANUAL,029031601h,TEMPERATURA DEL AIRE,°C,INST,HORAS,2024-03-25 15:56:51.663826,2025-04-30 16:05:53.495120,234.0


# 4 Consulta de datos de estaciones convencionales en el módulo de tiempo real.

**Módulo de tiempo real. __paramsRT__** tiene  acceso a los datos de las observaciones en tiempo real, a menor resolución temporal posible para cada estación (5 minutos en el caso de estaciones automáticas que trasnmiten por GRPS y 1 hora las que transmiten por GOES), se puede consultar continuamente y hasta tres meses hacia atras de la fecha actual

## Ejemplo 1. Consulta de parametros disponibles por estación
La siguiente celda muestra los parametros diponibles para cada estación, y la fecha del último dato de las estaciones convencionales y automáticas.

In [5]:
consulta = "select id_estacion,codigo_inamhi, nemonico, captor ,fecha_primer_dato ,fecha_ultimo_dato, fecha_ultima_act from public.estaciones_parametros;"
df = con_db.query_to_dataframe(consulta, params=con_db.paramsRT)
display(df)

,id_estacion,codigo_inamhi,nemonico,captor,fecha_primer_dato,fecha_ultimo_dato,fecha_ultima_act
0,62385,H1136,153041601h,ELECTROMECANICA,NaT,NaT,2025-05-20 21:58:20.719000
1,63937,M5021,021201601h,ELECTROMECANICA,2024-07-01 09:00:00,2025-06-26 06:00:00,2025-06-26 17:40:01.683116
2,63941,M5025,021201601h,ELECTROMECANICA,2024-07-01 09:00:00,2025-06-26 07:00:00,2025-06-26 17:40:01.683116
3,63942,M5026,021201601h,ELECTROMECANICA,2024-07-01 09:00:00,2025-06-26 05:00:00,2025-06-26 17:40:01.683116
4,63943,M5027,021201601h,ELECTROMECANICA,2024-07-01 09:00:00,2025-06-26 07:00:00,2025-06-26 17:40:01.683116
...,...,...,...,...,...,...,...
12557,64426,M5184,039191601h,ELECTROMECANICA,2022-07-17 05:00:00,2025-06-26 16:00:00,2025-06-26 17:40:03.468493
12558,26,M0004,037110401h,ELECTROMECANICA,2014-07-21 22:00:00,2025-06-26 15:00:00,2025-06-26 17:40:04.110419
12559,26,M0004,009010401h,ELECTROMECANICA,2014-07-21 22:00:00,2025-06-26 15:00:00,2025-06-26 17:40:05.724376
12560,62564,M0162,021200201h,ELECTROMECANICA,2014-07-21 22:00:00,2025-06-26 16:00:00,2025-06-26 17:40:02.554626


## Ejemplo 2. Consulta los datos de precipitación de estaciones convencionales que transmiten por libreta electrónica.

En este ejemplo vamos a consultar las estaciones convencionales que transmitieron datos de precipitación dentro de las últimas 24 horas, en libreta electrónica

In [6]:
fechaactual = datetime.now() # hora actual 
# NOTA: la hora esta en formato UTC, por tanto para la hora local en base de datos se debe sumar 5 horas,
# para este caso solo sumamos 4 porque queremos una hora atras la consulta
numero_horas = 1
ultimodia = fechaactual - timedelta(hours=(numero_horas+5))
fc = ultimodia.strftime('%Y-%m-%d %H:00:00')
consulta = """ SELECT rr.id_estacion, es.cod_inamhi, rr.fecha_toma_dato, rr.fecha_ingreso
            FROM convencionales._029031601h rr, estaciones AS es where es.id = rr.id_estacion and rr.fecha_toma_dato > '2025-06-03 00:00'
            order by fecha_toma_dato DESC; """
df = con_db.query_to_dataframe(consulta, params=con_db.paramsRT)

df['fecha_toma_dato_local'] = df['fecha_toma_dato'] - timedelta(hours=5)
df['fecha_ingreso_local'] = df['fecha_ingreso'] - timedelta(hours=5)
display(df)


,id_estacion,cod_inamhi,fecha_toma_dato,fecha_ingreso,fecha_toma_dato_local,fecha_ingreso_local
0,63761,M0025,2025-09-02 18:15:43.577102,2025-09-02 18:18:06.467996,2025-09-02 13:15:43.577102,2025-09-02 13:18:06.467996
1,63770,M0003,2025-09-02 18:13:45.221656,2025-09-02 18:15:20.330112,2025-09-02 13:13:45.221656,2025-09-02 13:15:20.330112
2,63760,M0031,2025-09-02 18:09:56.238989,2025-09-02 18:11:19.751074,2025-09-02 13:09:56.238989,2025-09-02 13:11:19.751074
3,63771,M0024,2025-09-02 18:08:54.120688,2025-09-02 18:10:34.901691,2025-09-02 13:08:54.120688,2025-09-02 13:10:34.901691
4,63756,M1094,2025-09-02 15:12:11.071696,2025-09-02 15:13:33.986140,2025-09-02 10:12:11.071696,2025-09-02 10:13:33.986140
...,...,...,...,...,...,...
1084,64400,VM0010,2025-06-03 07:54:26.701597,2025-06-03 12:58:23.452886,2025-06-03 02:54:26.701597,2025-06-03 07:58:23.452886
1085,63761,M0025,2025-06-03 07:07:10.104862,2025-06-03 12:11:20.215494,2025-06-03 02:07:10.104862,2025-06-03 07:11:20.215494
1086,64393,VM0003,2025-06-03 06:56:01.493544,2025-06-03 12:00:16.204738,2025-06-03 01:56:01.493544,2025-06-03 07:00:16.204738
1087,64396,VM0006,2025-06-03 06:47:55.146934,2025-06-03 11:51:51.329249,2025-06-03 01:47:55.146934,2025-06-03 06:51:51.329249


In [3]:
fechaactual = datetime.now() # hora actual 
# NOTA: la hora esta en formato UTC, por tanto para la hora local en base de datos se debe sumar 5 horas,
# para este caso solo sumamos 4 porque queremos una hora atras la consulta
numero_horas = 24
ultimodia = fechaactual - timedelta(hours=(numero_horas+5))
fc = ultimodia.strftime('%Y-%m-%d %H:00:00')
consulta = """ SELECT tem.id_estacion, es.cod_inamhi, tem.fecha_toma_dato, tem.fecha_ingreso, tem.term_hmd AS tem_humedo, tem.term_seco AS tem_seco 
FROM convencionales._029031601h tem, estaciones AS es where es.id = tem.id_estacion and tem.fecha_toma_dato > '2025-06-03 00:00'
order by fecha_toma_dato DESC; """
df = con_db.query_to_dataframe(consulta, params=con_db.paramsRT)

df['fecha_toma_dato_local'] = df['fecha_toma_dato'] - timedelta(hours=5)
df['fecha_ingreso_local'] = df['fecha_ingreso'] - timedelta(hours=5)
display(df)


,id_estacion,cod_inamhi,fecha_toma_dato,fecha_ingreso,tem_humedo,tem_seco,fecha_toma_dato_local,fecha_ingreso_local
0,63771,M0024,2025-11-19 13:26:14.868380,2025-11-19 13:35:06.320705,12.50000,15.30000,2025-11-19 08:26:14.868380,2025-11-19 08:35:06.320705
1,63770,M0003,2025-11-19 13:00:25.050216,2025-11-19 13:02:59.991921,9.40000,10.00000,2025-11-19 08:00:25.050216,2025-11-19 08:02:59.991921
2,63756,M1094,2025-11-19 12:14:14.232191,2025-11-19 12:16:20.178978,10.60000,12.00000,2025-11-19 07:14:14.232191,2025-11-19 07:16:20.178978
3,63767,M0008,2025-11-19 12:12:22.233859,2025-11-19 12:25:56.563457,18.40000,19.00000,2025-11-19 07:12:22.233859,2025-11-19 07:25:56.563457
4,63761,M0025,2025-11-19 12:11:32.450259,2025-11-19 12:13:09.523536,22.00000,22.40000,2025-11-19 07:11:32.450259,2025-11-19 07:13:09.523536
...,...,...,...,...,...,...,...,...
2740,64400,VM0010,2025-06-03 07:54:26.701597,2025-06-03 12:58:23.452886,None,21.20000,2025-06-03 02:54:26.701597,2025-06-03 07:58:23.452886
2741,63761,M0025,2025-06-03 07:07:10.104862,2025-06-03 12:11:20.215494,21.80000,22.80000,2025-06-03 02:07:10.104862,2025-06-03 07:11:20.215494
2742,64393,VM0003,2025-06-03 06:56:01.493544,2025-06-03 12:00:16.204738,None,17.24000,2025-06-03 01:56:01.493544,2025-06-03 07:00:16.204738
2743,64396,VM0006,2025-06-03 06:47:55.146934,2025-06-03 11:51:51.329249,None,17.70000,2025-06-03 01:47:55.146934,2025-06-03 06:51:51.329249


# 5 Consulta de datos de estaciones automáticas en el módulo histórico.

**Módulo de datos históricos.** ***paramsHIST*** contiene los datos hostóricos de las estaciones meteorólogicas existentes, aqui se almacena toda la información observada historicamente, a pesar de que no hay limite en el periodo de consulta se debe tener en cuenta las capacidades de computo asignasdas a cada usuario.

## 5.1 Ejemplo 1. Consulta de parametros disponibles por estación
La siguiente celda muestra los parametros diponibles para cada estación, y la fecha del último dato de las estaciones convencionales y automáticas.

In [23]:
consultaph = f"""select id_estacion, codigo_inamhi, captor, nemonico, numero_datos, 
descripcion_parametro, fecha_primer_dato, fecha_ultimo_dato, 
fecha_ultima_act from administrativo.estaciones_parametros;"""
dfph = con_db.query_to_dataframe(consultaph, params=con_db.paramsHIST)
dfph

,id_estacion,codigo_inamhi,captor,nemonico,numero_datos,descripcion_parametro,fecha_primer_dato,fecha_ultimo_dato,fecha_ultima_act
0,63800,H0854,ELECTROMECANICA,014100405m,0.0,NIVEL DEL AGUA m PROM 5 MINUTOS,NaT,NaT,2025-05-20 21:42:51.347306
1,4,M1170,ELECTROMECANICA,025200401h,7938.0,RADIACION UVE W/m² PROM 1 HORAS,2024-07-01 04:00:00,2025-05-31 03:00:00,2025-05-31 04:00:01.359731
2,63920,M1273,ELECTROMECANICA,022200401h,3983.0,RADIACION SOLAR REFLEJADA W/m² PROM 1 HORAS,2024-07-01 04:00:00,2024-12-16 14:00:00,2025-05-31 04:00:03.383429
3,64426,M5184,ELECTROMECANICA,022200401h,247.0,RADIACION SOLAR REFLEJADA W/m² PROM 1 HORAS,2025-05-20 21:00:00,2025-05-31 03:00:00,2025-05-31 04:00:03.383429
4,64387,HM004,ELECTROMECANICA,143121601h,6003.0,COLOR PT CO mg/L INST 1 HORAS,2024-08-06 20:00:00,2025-05-31 02:00:00,2025-05-31 04:00:01.493956
...,...,...,...,...,...,...,...,...,...
10639,63804,M5134,ELECTROMECANICA,009010101m,0.0,HUMEDAD RELATIVA DEL AIRE % RH MAX 1 MINUTOS,NaT,NaT,2025-02-19 17:43:30.972013
10640,63804,M5134,ELECTROMECANICA,009010201m,0.0,HUMEDAD RELATIVA DEL AIRE % RH MIN 1 MINUTOS,NaT,NaT,2025-02-19 17:43:30.975011
10641,63804,M5134,ELECTROMECANICA,009013201m,0.0,HUMEDAD RELATIVA DEL AIRE % RH DEV 1 MINUTOS,NaT,NaT,2025-02-19 17:43:30.977932
10642,63804,M5134,ELECTROMECANICA,018070401m,0.0,PRESION ATMOSFERICA hPa PROM 1 MINUTOS,NaT,NaT,2025-02-19 17:43:30.980970


## 5.2 Ejemplo 2 Parte1. Consulta de datos de estaciones automaticas de la ùltima hora
La siguiente como consultar dastos de precipitación todas las estaciones para la última hora, se consultará únicamente las estaciones automáticas.

In [3]:
#### Generamos el listado de estaciones por propietario
id_pro = (12,26)  

query_propietario = f"""
    select codigo, punto_obs, propietario, id_estacion, captor, categoria, latitud, longitud, altitud
from administrativo.vta_estaciones where id_propietario in {id_pro} order by codigo;
"""
#print(query_propietario)
# Ejecutar la consulta y almacenar los resultados en un DataFrame
df_propietarios = con_db.query_to_dataframe(query_propietario)
# Mostrar los primeros registros del DataFrame
#print(df_propietarios.head(3))

ids_estaciones = df_propietarios.iloc[:,0]
ids_estaciones = tuple(ids_estaciones)
#print(ids_estaciones)

### cambiar por la nombre del nemònico que se desea consultar 
# ***********************
nemonico="017140801h"

ultimaHora = datetime.now() # hora actual 
#print(ultimaHora)
# NOTA: la hora esta en formato UTC, por tanto para la hora real en base de datos se debe sumar 5 horas,
# para este caso solo sumamos 4 porque queremos una hora atras la consulta
ultimaHora = ultimaHora + timedelta(hours=4)
formateado = ultimaHora.strftime('%Y-%m-%d %H:00:00')
#print(formateado)
consultauh = f"""SELECT rr.id_estacion, es.codigo, es.punto_obs,
rr.fecha_toma_dato, rr."1h" AS precipitacion 
FROM automaticas._{nemonico} rr, administrativo.vta_estaciones AS es
where es.id_estacion = rr.id_estacion 
and rr.fecha_toma_dato = '{formateado}' and es.codigo in {ids_estaciones}
order by rr.fecha_toma_dato, es.codigo;"""
#print(consultauh)
dfuh = con_db.query_to_dataframe(consultauh, params=con_db.paramsHIST)
#cambiamos la hora a local UTC-5
#dfuh['fecha_toma_dato'] = pd.to_datetime(dfuh['fecha_toma_dato'])
dfuh['fecha_toma_dato'] = dfuh['fecha_toma_dato'] - timedelta(hours=5)
# limpiamos el dataframe de valore None en la columna precipitación 
dfuh = dfuh.dropna(subset=['precipitacion']).reset_index(drop=True)
dfuh

,id_estacion,codigo,punto_obs,fecha_toma_dato,precipitacion
0,63937,M5021,SALVEFACHA-OYACACHI,2025-06-11 08:00:00,0.00
1,63941,M5025,LA VIRGEN-PAPALLACTA,2025-06-11 08:00:00,0.00
2,63943,M5027,SAN MARCOS-ILINIZAS (EL CHAUPI),2025-06-11 08:00:00,0.00
3,63946,M5030,GORDILLO (PINTAG),2025-06-11 08:00:00,0.00
4,63947,M5031,CHUMILLOS (CANGAHUA),2025-06-11 08:00:00,0.00
5,64430,M5044,YARUQUI -EPMAPS QUITO,2025-06-11 08:00:00,0.00
6,64437,M5050,EL TROJE - EPMAPS QUITO,2025-06-11 08:00:00,0.00
7,66260,M5056,TOCTIUCO - EPMAPS QUITO,2025-06-11 08:00:00,0.00
8,66262,M5058,IZOBAMBA - EPMAPS QUITO,2025-06-11 08:00:00,0.00
9,64438,M5060,EL CINTO,2025-06-11 08:00:00,0.00


## 5.3 Ejemplo 3 Parte 2. Consulta de datos de estaciones automaticas de las últimas 24 horas filtrando por propietario

En esta sección se aplicara primero un filtro para obtener un grupo de estaciones por propietario, en este ejemplo para id_propietario 12, de EPMAPS y FONAG.





In [2]:
#### Generamos el listado de estaciones por propietario
id_pro = (12,26)  

query_propietario = f"""
    select codigo, punto_obs, propietario, id_estacion, captor, categoria, latitud, longitud, altitud
from administrativo.vta_estaciones where id_propietario in {id_pro} order by codigo;
"""
#print(query_propietario)
# Ejecutar la consulta y almacenar los resultados en un DataFrame
df_propietarios = con_db.query_to_dataframe(query_propietario)
# Mostrar los primeros registros del DataFrame
#print(df_propietarios.head(3))

ids_estaciones = df_propietarios.iloc[:,0]
ids_estaciones = tuple(ids_estaciones)
#print(ids_estaciones)
### cambiar por la nombre del nemònico que se desea consultar 
# ***********************
nemonico="017140801h"
numero_horas = 24
# Representa el calculo que se debe realizar en la columna agregado
agregado = 0 ## 0 para suma, 1 para promedio 
ultimaHora = datetime.now() # hora actual 
# NOTA: la hora esta en formato UTC, por tanto para la hora local en base de datos se debe sumar 5 horas,
# para este caso solo sumamos 4 porque queremos una hora atras la consulta
ultimaHora = ultimaHora - timedelta(hours=(numero_horas+4))
fc = ultimaHora.strftime('%Y-%m-%d %H:00:00')
hora = ultimaHora.strftime('%H')
print("fecha de inicio de la consulta"," - ",fc)
for i in range(numero_horas):
    consultauh = f"""SELECT rr.id_estacion, es.codigo, rr."1h" AS precipitacion 
    FROM automaticas._{nemonico} rr, administrativo.vta_estaciones AS es
    where es.id_estacion = rr.id_estacion and rr.fecha_toma_dato = '{fc}' and es.codigo in {ids_estaciones}
    order by rr.fecha_toma_dato, es.codigo;"""
    dfuh = con_db.query_to_dataframe(consultauh, params=con_db.paramsHIST)
    cod = f'codigo_{i+1}h'
    #pre = f"pre_{i+1}h"
    pre = ultimaHora.strftime('%m_%d_%H')
    dfuh.rename(columns={'codigo': cod,'precipitacion': pre }, inplace=True)
    #Unir un solo dataframe
    if i == 0:
        df_acum = dfuh
    else: 
        df_acum = pd.merge(df_acum, dfuh, on='id_estacion',how='outer')
    
    ultimaHora = ultimaHora + timedelta(hours = + 1)
    fc = ultimaHora.strftime('%Y-%m-%d %H:00:00')

df_acum


fecha de inicio de la consulta  -  2025-06-10 05:00:00


,id_estacion,codigo_1h,06_10_05,codigo_2h,06_10_06,codigo_3h,06_10_07,codigo_4h,06_10_08,codigo_5h,06_10_09,codigo_6h,06_10_10,codigo_7h,06_10_11,codigo_8h,06_10_12,codigo_9h,06_10_13,codigo_10h,06_10_14,codigo_11h,06_10_15,codigo_12h,06_10_16,codigo_13h,06_10_17,codigo_14h,06_10_18,codigo_15h,06_10_19,codigo_16h,06_10_20,codigo_17h,06_10_21,codigo_18h,06_10_22,codigo_19h,06_10_23,codigo_20h,06_11_00,codigo_21h,06_11_01,codigo_22h,06_11_02,codigo_23h,06_11_03,codigo_24h,06_11_04
0,63937,M5021,0.00,M5021,0.00,M5021,0.00,M5021,0.00,M5021,1.60,M5021,0.40,M5021,3.40,M5021,1.20,M5021,1.10,M5021,0.30,M5021,0.30,M5021,0.50,M5021,0.10,M5021,0.30,M5021,0.30,M5021,0.10,M5021,0.00,M5021,0.00,M5021,0.00,M5021,0.00,M5021,0.00,M5021,0.00,M5021,0.00,M5021,0.00
1,63941,M5025,0.00,M5025,0.00,M5025,0.00,M5025,0.00,M5025,0.00,M5025,0.50,M5025,1.80,M5025,0.80,M5025,0.60,M5025,1.00,M5025,0.10,M5025,0.20,M5025,0.30,M5025,0.20,M5025,0.80,M5025,0.20,M5025,0.00,M5025,0.10,M5025,0.00,M5025,0.00,M5025,0.00,M5025,0.00,M5025,0.00,M5025,0.00
2,63942,M5026,0.00,M5026,0.00,M5026,0.00,M5026,0.00,M5026,0.00,M5026,1.00,M5026,1.60,M5026,0.90,M5026,0.00,M5026,0.00,M5026,0.00,M5026,0.00,M5026,0.00,M5026,0.10,M5026,0.00,M5026,0.00,M5026,0.00,M5026,0.00,M5026,0.00,M5026,0.00,M5026,0.00,M5026,0.00,M5026,0.00,M5026,0.00
3,63943,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.50,M5027,0.90,M5027,2.30,M5027,0.20,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00,M5027,0.00
4,63944,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.30,M5028,0.60,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00,M5028,0.00
5,63945,M5029,0.00,M5029,0.00,M5029,0.00,M5029,0.00,M5029,0.00,M5029,0.50,M5029,0.80,M5029,1.60,M5029,0.10,M5029,0.10,M5029,0.00,M5029,0.10,M5029,0.00,M5029,0.00,M5029,0.00,M5029,0.00,M5029,0.00,M5029,0.00,M5029,0.00,M5029,0.00,M5029,0.00,M5029,0.00,M5029,0.00,M5029,0.00
6,63946,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.20,M5030,0.00,M5030,0.90,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.30,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.00,M5030,0.00
7,63947,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.10,M5031,1.70,M5031,0.80,M5031,4.90,M5031,1.40,M5031,0.30,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.00,M5031,0.00
8,63949,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.70,M5075,0.40,M5075,1.20,M5075,0.10,M5075,0.30,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00,M5075,0.00
9,63951,M5077,0.00,M5077,0.00,M5077,0.00,M5077,0.00,M5077,0.00,M5077,0.00,M5077,0.20,M5077,0.80,M5077,2.50,M5077,0.10,M5077,0.00,M5077,0.00,M5077,0.00,M5077,0.40,M5077,0.00,M5077,0.00,M5077,0.00,M5077,0.00,M5077,0.00,M5077,0.00,M5077,0.00,M5077,0.00,M5077,0.00,M5077,0.00
